<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter_em_contrucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [31]:
!pip install fredapi
!pip install pycountry

from fredapi import Fred
import pycountry
import pandas as pd
from datetime import datetime, timedelta
import requests
from collections import defaultdict

# Inicialize a chave do FRED
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')


# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

## Importar dados

In [32]:
# Importar dados PPP
url = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/eabb3ca9caaca37dc9d9f1ae46ad03f7bc699954/ppp_att.csv'
ppp_data =  pd.read_csv(url, delimiter=',')

## Obter taxa de conversão do dólar 2024

In [33]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Mudar ISO3 para ISO2
def iso3_to_iso2(iso3):
    try:
    # Convert iso3 to string and ensure it's uppercase
        iso3 = str(iso3).upper()
        return pycountry.countries.get(alpha_3=iso3).alpha_2
    except (LookupError, AttributeError):
        # Handle both LookupError and AttributeError
        return None

# Importar taxa de câmbio dos países
url2 = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/refs/heads/main/EXCHANGE_RATES.csv'
exchange_rates24 =  pd.read_csv(url2)
exchange_rates24 = exchange_rates24[['Country Code', '2024 [2024]']]
exchange_rates24 = exchange_rates24.rename(columns={'2024 [2024]': '2024'})
exchange_rates24['Country Code'] = exchange_rates24['Country Code'].apply(iso3_to_iso2)

## Função Conversão

In [34]:
# Função para obter o PPP ajustado de um país em um ano específico a partir do CSV
def get_ppp_adjusted(country_code, year):
    ppp_row = ppp_data[(ppp_data['Country ID'] == country_code) & (ppp_data['Year'] == year)]
    return ppp_row.iloc[0]['PPP']

# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
        if row['Ano'] == 2024:
            # Procura o exchange rate diretamente no dataframe exchange_rates24
            exchange_rate_row = exchange_rates24[exchange_rates24['Country Code'] == row['País_Moeda']]
            if not exchange_rate_row.empty:
                exchange_rate = exchange_rate_row.iloc[0]['2024']  # Obtém a taxa de câmbio para 2024
                exchange_rate = float(exchange_rate)
                gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate
                return gasto_em_dolar_2024
        else:
            # Para demais anos
            ppp_adjusted = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
            gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

            # Ajuste pela inflação dos EUA
            cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
            gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    elif row['Ano2'] == 2024:
        # Calcular gasto via taxa de câmbio de 2024
        exchange_rate_row = exchange_rates24[exchange_rates24['Country Code'] == row['País_Moeda']]
        if not exchange_rate_row.empty:
            exchange_rate = float(exchange_rate_row.iloc[0]['2024'])
            gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate

            # Calcular também via PPP + inflação, considerando apenas Ano1
            ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
            gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_ano1

            cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
            gasto_ajustado_ppp = gasto_em_dolar_ppp * (cpi_atual / cpi_ano1)

            # Média entre os dois métodos
            gasto_ajustado = (gasto_em_dolar_2024 + gasto_ajustado_ppp) / 2

    else:
        # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2
        ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
        ppp_ano2 = get_ppp_adjusted(row['País_Moeda'], row['Ano2'])

        # Média dos gastos entre Ano1 e Ano2
        gasto_em_dolar_ppp = row['Gasto_Local'] / ((ppp_ano1 + ppp_ano2) / 2)

        # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2
        cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
        cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

        # Calcula a média do CPI para ajuste pela inflação
        cpi_media = (cpi_ano1 + cpi_ano2) / 2

        # Ajusta o gasto usando a média do CPI
        gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


def agrupar_gastos(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_agrupado = df.groupby(['Período', 'País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_agrupado

def agrupar_gastos_pais(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_pais = df.groupby(['País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_pais

def agrupar_gastos_programa(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_prog = df.groupby(['Programa', 'País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_prog

def agrupar_fontes_programa(df):
    # Agrupa por 'Programa' e concatena fontes únicas (sem repetição)
    df_prog = df.groupby('Programa')['Fontes'].apply(lambda x: '; '.join(sorted(set(x.astype(str))))).reset_index()
    return df_prog

def agrupar_fontes_pais(df):
    # Agrupa por 'País do Programa' e concatena fontes únicas (sem repetição)
    df_pais = df.groupby('País do Programa')['Fontes'].apply(lambda x: '; '.join(sorted(set(x.astype(str))))).reset_index()
    return df_pais

def agrupar_fontes_periodo_pais(df):
    # Agrupa por 'Período' e 'País do Programa', concatenando fontes únicas (sem repetição)
    df_periodo_pais = df.groupby(['Período', 'País do Programa'])['Fontes'] \
        .apply(lambda x: '; '.join(sorted(set(x.astype(str))))).reset_index()
    return df_periodo_pais

## Função Formatação

In [35]:
# Função para formatar valores em milhões, bilhões ou trilhões para Moeda Local
def formatar_valor_moeda_local(valor, codigo_pais):
    currency_code = get_currency_code(codigo_pais)  # Obtém o código da moeda do país
    if not currency_code:
        currency_code = "Moeda desconhecida"  # Se não houver código, exibe "Moeda desconhecida"

    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{currency_code} {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{currency_code} {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{currency_code} {valor / 1_000_000:.2f} milhões"
    else:
        return f"{currency_code} {valor:.2f}"

# Função para formatar valores em milhões, bilhões ou trilhões para Dólar Ajustado
def formatar_valor_dolar(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"USD {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"USD {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"USD {valor / 1_000_000:.2f} milhões"
    else:
        return f"USD {valor:.2f}"

# Função para juntar Ano com Ano2
def definir_periodo(df):
    # Verifica se as colunas Ano e Ano2 existem no DataFrame
    if 'Ano' in df.columns and 'Ano2' in df.columns:
        # Aplica a condição para criar a coluna Período
        df['Período'] = df.apply(lambda row: str(row['Ano']) if row['Ano2'] == 0 else f"{row['Ano']} a {row['Ano2']}", axis=1)
    else:
        print("Colunas 'Ano' e 'Ano2' não encontradas no DataFrame.")
    return df

## Insira seus dados:

In [36]:
dados = {
    'Programa': [
        "Healthy China 2030",                                           # 1
        "Advanced Manufacturing Plan",                                  # 2
        "Advanced Manufacturing Plan",                                  # 3
        "Build Back Better: Our Plan for Growth and Innovation",          # 4
        "Build Back Better: Our Plan for Growth and Innovation",          # 5
        "Industrial Strategy Challenge Fund: Healthy ageing",           # 6
        "Industrial Strategy Challenge Fund: Healthy ageing",           # 7
        "Industrial Strategy Challenge Fund: Leading-edge healthcare",    # 8
        "Industrial Strategy Challenge Fund: Leading-edge healthcare",    # 9
        "Industrial Strategy Challenge Fund: Leading-edge healthcare",    # 10
        "Life Sciences 2030 Skills Strategy",                             # 11
        "Life Sciences 2030 Skills Strategy",                             # 12
        "Life Sciences Innovative Manufacturing Fund",                    # 13
        "Life Sciences Vision",                                           # 14
        "Life Sciences Vision",                                           # 15
        "Life Sciences Vision",                                           # 16
        "Life Sciences Vision",                                           # 17
        "Medicines Manufacturing Challenge",                              # 18
        "Medicines Manufacturing Industry Partnership (MMIP)",            # 19
        "The Life Sciences Industrial Strategy",                          # 20
        "The Life Sciences Industrial Strategy",                          # 21
        "The Life Sciences Industrial Strategy",                          # 22
        "The Life Sciences Industrial Strategy",                          # 23
        "The Life Sciences Industrial Strategy",                          # 24
        "The Life Sciences Industrial Strategy",                          # 25
        "UK Innovation Strategy",                                         # 26
        "UK Innovation Strategy",                                         # 27
        "Assistance to Pharmaceutical Industry for Common Facilities (APICF)",  # 28
        "BIRAC SEED Fund",                                                # 29
        "BIRAC SPARSH",                                                   # 30
        "BIRAC SPARSH",                                                   # 31
        "BIRAC SPARSH",                                                   # 32
        "BIRAC SPARSH",                                                   # 33
        "BIRAC SPARSH",                                                   # 34
        "BIRAC SPARSH",                                                   # 35
        "Cluster Development Programme for Pharma Sector (CDP-PS)",         # 36
        "Cluster Development Programme for Pharma Sector (CDP-PS)",         # 37
        "Credit Linked Capital Subsidy (CLCS)",                           # 38
        "National Policy on R&D and Innovation in the Pharma-MedTech Sector",# 39
        "Pharmaceutical & Medical Devices Promotion and Development Scheme (PMPDS)",  # 40
        "Pharmaceuticals Technology Upgradation Assistance Scheme (PTUAS)",   # 41
        "Production Linked Incentive (PLI) Scheme for promotion of domestic manufacturing of critical Key Starting Materials (KSMs)/ Drug Intermediates(DIs) and Active Pharmaceutical Ingredients (APIs)",  # 42
        "Production Linked Incentive Scheme (PLI) for Pharmaceuticals",      # 43
        "Production Linked Incentive Scheme (PLI) for Pharmaceuticals",      # 44
        "Production Linked Incentive Scheme (PLI) for Pharmaceuticals",      # 45
        "Scheme for Development of Pharmaceutical Industry (2019)",         # 46
        "Scheme for Promotion of Bulk Drug Parks",                         # 47
        "Strengthening of Pharmaceutical Industry Scheme",                # 48
        "Strengthening of Pharmaceutical Industry Scheme",                # 49
        "Bio-Health Industry Innovation Strategy",                        # 50
        "Bio-Health Industry Innovation Strategy",                        # 51
        "Bio-Health Industry Innovation Strategy",                        # 52
        "Bio-Health Industry Innovation Strategy",                        # 53
        "Bio-Health Industry Innovation Strategy",                        # 54
        "Comprehensive Plan to Promote the Biopharma Industry",           # 55
        "First National Strategic Technology Nurture Plan",               # 56
        "K-Bio Vaccine Fund",                                               # 57
        "K-Bio Vaccine Fund",                                               # 58
        "Korean ARPA-H",                                                    # 59
        "Korean ARPA-H",                                                    # 60
        "MSS - Ministry of SMEs and Startups",                              # 61
        "Strategies for Creating New Bio-Health Markets",                   # 62
        "Strategies for Creating New Bio-Health Markets",                   # 63
        "Strategies for Creating New Bio-Health Markets",                   # 64
        "Strategies for Creating New Bio-Health Markets",                   # 65
        "Strategies for Creating New Bio-Health Markets",                   # 66
        "Strategies for Creating New Bio-Health Markets",                   # 67
        "Tech Incubation Program for Start-ups (TIPS)",                     # 68
        "Tech Incubation Program for Start-ups (TIPS)",                     # 69
        "Tech Incubation Program for Start-ups (TIPS)",                     # 70
        "Tech Incubation Program for Start-ups (TIPS)",                     # 71
        "Tech Incubation Program for Start-ups (TIPS)",                     # 72
        "Tech Incubation Program for Start-ups (TIPS)",                     # 73
        "Programa Especial de Ciencia, Tecnología e Innovación 2021-2024",    # 74
        "Programa Especial de Ciencia, Tecnología e Innovación 2021-2024",    # 75
        "Programa Sectorial de Salud 2020-2024",                            # 76
        "Programa Sectorial de Salud 2020-2024",                            # 77
        "Programa Sectorial de Salud 2020-2024",                            # 78
        "ARPA-H",                                                         # 79
        "ARPA-H",                                                         # 80
        "Bold Goals for U.S. Biotechnology and Biomanufacturing",           # 81
        "Defense Production Act (DPA)",                                   # 82
        "Defense Production Act (DPA)",                                   # 83
        "EOs 14017 and 14001",                                              # 84
        "EOs 14017 and 14001",                                              # 85
        "Executive Order on Advancing Biotechnology and Biomanufacturing Innovation",  # 86
        "Executive Order on Advancing Biotechnology and Biomanufacturing Innovation",  # 87
        "National Biodefense Strategy",                                     # 88
        "National Biodefense Strategy",                                     # 89
        "National Biodefense Strategy",                                     # 90
        "National Biodefense Strategy",                                     # 91
        "Operation Warp Speed (OWS)",                                       # 92
        "OperationWarp Speed (OWS)",                                        # 93
        "Reimagined Industrial Strategy",                                   # 94
        "Reimagined Industrial Strategy",                                   # 95
        "Reimagined Industrial Strategy",                                   # 96
        "Strategic Health Innovation Partnership"                           # 97
    ],
    'Descrição': [
        "Gasto governamental em P&D em biotecnologia*",                    # 1
        "Montante total disponibilizado para financiamento de diferentes setores, incluindo o de Ciências da Vida, por cinco anos (2025-2030)**",  # 2
        "Parte do plano destinada às indústrias das ciências da vida**",     # 3
        "Não específico para indústria da saúde: investimento governamental em P&D em 2021-22**",  # 4
        "Não específico para indústria da saúde: financiamento para a Advanced Research & Invention Agency (ARIA)**",  # 5
        "Investimento no UK Dementia Research Institute (UKDRI)**",         # 6
        "Investimento em centros regionais de excelência e um programa de uso de dados para medicina de precisão e diagnósticos preventivos**",  # 7
        "Financiamento para 3 centros de tratamento de terapias avançadas**",  # 8
        "Desafio para desenvolvimento de novas medicinas, vacinas e outros produtos da saúde, a ser investido em 4 anos**",  # 9
        "Investimento no Vaccines Manufacturing Innovation Centre**",      # 10
        "Impulsionar o crescimento e a inovação no setor de ciências da vida**",  # 11
        "Pacote de financiamento para manufatura em ciências biológicas entre 2025-2030**",  # 12
        "Capital para investimento na produção de medicamentos e tecnologia médica. Primeira Fase**",  # 13
        "NHS AI Health and Care Award entregue pelo NHSX AI Lab em parceria com o AAC*",  # 14
        "Lançamento do Life Sciences Investment Programme (LSIP)**",        # 15
        "Aumento da capacidade de manufatura de vacinas*",                   # 16
        "Fundo de apoio à manufatura**",                                    # 17
        "Investimento no desenvolvimento de tecnologias inovadoras para fabricar medicamentos e acelerar o acesso dos pacientes a novos medicamentos e tratamentos*",  # 18
        "Follow the Green, High-Tech Road**",                              # 19
        "Aumentar investimento em P&D, apoiar medidas para o crescimento da manufatura de medicinas e outras ações**",  # 20
        "Aumento em investimentos em P&D para os próximos 10 anos**",        # 21
        "Atingir esse nível de gasto governamental em P&D em 2027**",         # 22
        "Gasto governamental em P&D em 2017*",                              # 23
        "Fundo para acesso a medicamentos e investimento em ciências da vida**",  # 24
        "Centros de terapias avançadas, apoio à manufatura de medicamentos e qualificação profissional*",  # 25
        "Financiamento para o crescimento de empresas de ciências da vida e em P&D e manufatura no setor.**",  # 26
        "Investimentos para promover saúde por meio da inovação em produtos e serviços; prever doenças e desafios globais; prevenir ameaças à saúde e à agricultura; fornecer gestão e medicamentos transformadores; e produzir alimentos e medicamentos sustentáveis e eficientes.**",  # 27
        "Montante total investido em três projetos para fortalecer os clusters de MSME farmacêuticas existentes*",  # 28
        "SEED Fund apoiou 112 startups com esse investimento de equity total*",  # 29
        "Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2017-2018*",  # 30
        "Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2018-2019*",  # 31
        "Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2019-2020*",  # 32
        "Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2020-2021*",  # 33
        "Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2021-2022*",  # 34
        "Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2022-2023*",  # 35
        "Orçamento durante o período de 2018 a 2020 (50% Promoção de Parques de IFAs + 50% assistência para parques de dispositivos médicos)**",  # 36
        "Orçamento ampliado em 2023 até o ano fiscal de 2025-2026**",          # 37
        "Desembolso**",                                                     # 38
        "Sob essa política, foi lançado o esquema Promotion of Research and Innovation in Pharma and MedTech Sector (PRIP), com este orçamento.**",  # 39
        "Desembolso financeiro entre 2021/2022 e 2025/2026**",               # 40
        "Orçamento para o período de 2018 a 2020**",                         # 41
        "Desembolso financeiro para o período de produção sob o esquema, que é do ano fiscal de 2022-23 ao ano fiscal de 2028-29.**",  # 42
        "Incentivo para produção doméstica de materiais de partida, intermediários de síntese e IFAs",  # 43
        "Programa para medicamentos**",                                     # 44
        "Materiais críticos de partida/Intermediários de Medicamentos e Ingredientes Farmacêuticos Ativos – 47 solicitações aprovadas**",  # 45
        "Orçamento destinado a apoiar a indústria farmacêutica nacional, com foco em centros de facilidades comuns, atualização tecnológica e promoção do setor.**",  # 46
        "Orçamento de 2020-21 até 2024-25. Busca estabelecer três parques de produção de IFAs e intermediários de medicamentos com infraestrutura comum avançada.**",  # 47
        "Orçamento para o período de 2021-2026**",                             # 48
        "O governo indiano planejava gastar no programa National Policy on R&D and Innovation in the PharmaMedTech Sector in India**",  # 49
        "Antes da estratégia: investimentos em P&D na biotecnologia e no setor da saúde em 2017*",  # 50
        "Meta de investimentos em P&D na biotecnologia e no setor da saúde para 2025**",  # 51
        "Empréstimos de políticas por ano pelos próximos cinco anos**",      # 52
        "O governo planeja investir no desenvolvimento de dois projetos interagências – desenvolvimento de novos medicamentos e tecnologia de medicina regenerativa. Os dois projetos nacionais fazem parte da Bio-Health Industry Innovation Strategy**",  # 53
        "Investimento na área de medicina regenerativa por 10 anos a partir de 2021**",  # 54
        "Será investido no K-Bio Vaccine Fund e, com base na análise dos resultados, outro mega fundo será criado**",  # 55
        "De 2023 a 2028**",                                                 # 56
        "A futura tecnologia médica será financiada através da criação do K-Bio Vaccine Fund, no primeiro semestre de 2023, para expandir o investimento no desenvolvimento de vacinas e terapêuticos.**",  # 57
        "K-Bio Vaccine Fund Nos. 1 e 2 arrecadaram este valor em investimentos.*",  # 58
        "Alocado**",                                                       # 59
        "Entre 2024-2033**",                                                # 60
        "Orçamento para o ministério em 2020*",                             # 61
        "O National New Drug Development Project continuará a receber investimentos entre 2021 e 2030**",  # 62
        "Fortalecimento de P&D e Inovação em Bio-saúde**",                   # 63
        "Desenvolvimento de Infraestrutura e Apoio a Startups Bio-saúde**",    # 64
        "Criação de Estruturas Jurídicas e Infraestrutura para Exportação: ampliação do financiamento para Seguro de Exportação, Banco de Exportação-Importação da Coreia, Fundo de Garantia de Crédito da Coreia, Financiamento de Tecnologia da Coreia e Banco de Desenvolvimento da Coreia e Banco Industrial da Coreia**",  # 65
        "A Fase 1 do projeto Interdepartmental Full-Cycle Medical Device R&D será executada entre 2020 e 2025**",  # 66
        "Orçamento acumulado desde 2018 ao programa de suporte completo do Bio-Health Innovation Startup Center*",  # 67
        "Orçamento TIPS em 2017*",                                           # 68
        "Orçamento TIPS em 2018*",                                           # 69
        "Orçamento TIPS em 2019*",                                           # 70
        "Orçamento TIPS em 2020*",                                           # 71
        "Orçamento TIPS em 2021*",                                           # 72
        "Orçamento TIPS em 2022*",                                           # 73
        "Meta para 2024 em gasto federal em ciência e tecnologia em temas prioritários (um deles é Saúde)**",  # 74
        "Meta para 2024 em gasto federal em pesquisa científica e desenvolvimento experimental em temas prioritários (um deles é Saúde)**",  # 75
        "Investimento governamental diretamente executado no âmbito do programa. com foco no sistema de saúde como um todo (impactando a industria da saúde): 2020*",  # 76
        "Investimento governamental diretamente executado no âmbito do programa. com foco no sistema de saúde como um todo (impactando a industria da saúde): 2021*",  # 77
        "Investimento governamental diretamente executado no âmbito do programa. com foco no sistema de saúde como um todo (impactando a industria da saúde): 2022*",  # 78
        "Para o ano fiscal de 2022, este foi o orçamento solicitado pelo presidente para serem gastos ao longo de três anos. **",  # 79
        "Com leis de 2022 e 2023, foi concedido um adicional para projetos de alto risco e alto impacto, para o fim de 2024 e 2025**",  # 80
        "Destinado pelo Departamento de Defesa dos EUA para apoiar a biotecnologia e a biomanufatura, impulsionando a inovação, a bioeconomia e a produção de biocombustíveis e bioprodutos renováveis.**",  # 81
        "Obrigou este valor à compra de fornecimentos e à reposição do Estoque Nacional Estratégico*",  # 82
        "Prêmio para projetos de expansão de produção doméstica de suprimentos médicos usando recursos do CARES Act**",  # 83
        "Valores gastos*",                                                  # 84
        "Investimentos planejados**",                                      # 85
        "Objetivo de melhorar e expandir a capacidade e processos de produção de biomanufatura doméstica**",  # 86
        "Valor investido pelo governo federal dos EUA para o aumento de capacidade de produção de seis medicamentos que sofriam de escassez.*",  # 87
        "Programa Beyond the Needle e Coalition for Epidemic Preparedness Innovations**",  # 88
        "Pedido do presidente para cinco anos para preparação para pandemias e biodefesa**",  # 89
        "Gastos relatados com biodefesa em todas as D / As para 2019*",         # 90
        "Orçamento pedido pelo HSS para implementar a National Biodefense Strategy. No entanto, o orçamento de 2020 não incluiu esse valor requisitado.**",  # 91
        "Valor dos contratos de candidatos a vacinas aprovados pela BARDA e outras agências federais*",  # 92
        "Valor dos contratos governamentais federais para suprimentos auxiliares de vacinas COVID-19*",  # 93
        "Financiamento industrial fornecido ao longo de cinco anos*",         # 94
        "Abertura da nova instalação Sappi-Saiccor, que processa madeira bruta em polpa que pode ser usada para viscose, produtos farmacêuticos e outras aplicações industriais.*",  # 95
        "A IDC facilitou atividades de localização e beneficiamento em setores como aço, produtos farmacêuticos, automotivo e alimentício*",  # 96
        "Financiamento inicial (seed funding) para apoiar projetos de pesquisa em saúde nas quatro principais instituições de pesquisa pública da província de Eastern Cape**"  # 97
    ],
    'Gasto_Local': [
         291000000000,   # 1
         4500000000,     # 2
         520000000,      # 3
         14600000000,    # 4
         800000000,      # 5
         40000000,       # 6
         280000000,      # 7
         21000000,       # 8
         181000000,      # 9
         66000000,       # 10
         650000000,      # 11
         520000000,      # 12
         17000000,       # 13
         140000000,      # 14
         200000000,      # 15
         350000000,      # 16
         20000000,       # 17
         255000000,      # 18
         1100000000,     # 19
         5000000000,     # 20
         80000000000,    # 21
         4000000000,     # 22
         3000000000,     # 23
         700000000,      # 24
         248100000,      # 25
         200000000,      # 26
         600000000,      # 27
         547300000,      # 28
         300000000,      # 29
         30120000,       # 30
         56422100,       # 31
         64280000,       # 32
         56640000,       # 33
         92100000,       # 34
         61300000,       # 35
         4000000000,     # 36
         3000000000,     # 37
         29000000000,    # 38
         50000000000,    # 39
         215000000,      # 40
         1440000000,     # 41
         69400000000,    # 42
         69400000000,    # 43
         150000000000,   # 44
         54000000000,    # 45
         4800000000,     # 46
         30000000000,    # 47
         5000000000,     # 48
         50000000000,    # 49
         2600000000000,  # 50
         4000000000000,  # 51
         2000000000000,  # 52
         2800000000000,  # 53
         595500000000,   # 54
         1000000000000,  # 55
         2100000000000,  # 56
         500000000000,   # 57
         261600000000,   # 58
         49500000000,    # 59
         1880000000000,  # 60
         849200000000,   # 61
         2200000000000,  # 62
         46650000000,    # 63
         272600000000,   # 64
         361300000000000,  # 65
         1200000000000,  # 66
         7100000000,     # 67
         84000000000,    # 68
         104200000000,   # 69
         145400000000,   # 70
         195100000000,   # 71
         208000000000,   # 72
         293400000000,   # 73
         109670000000,   # 74
         50220000000,    # 75
         74520000000,    # 76
         48510000000,    # 77
         40240000000,    # 78
         6500000000,     # 79
         2500000000,     # 80
         1000000000,     # 81
         8400000000,     # 82
         1510000000000,  # 83
         4460000000,     # 84
         2650000000,     # 85
         2000000000,     # 86
         105000000,      # 87
         156800000,      # 88
         88000000000,    # 89
         7500000000,     # 90
         5000000,        # 91
         18230000000,    # 92
         951800000,      # 93
         100000000000,   # 94
         7700000000,     # 95
         10000000000,    # 96
         4000000         # 97
    ],
    'Ano': [
         2019,   # 1
         2023,   # 2
         2023,   # 3
         2021,   # 4
         2021,   # 5
         2018,   # 6
         2018,   # 7
         2018,   # 8
         2018,   # 9
         2018,   # 10
         2022,   # 11
         2022,   # 12
         2023,   # 13
         2021,   # 14
         2021,   # 15
         2021,   # 16
         2021,   # 17
         2017,   # 18 (2017 à 2021)
         2023,   # 19
         2017,   # 20
         2017,   # 21
         2019,   # 22
         2017,   # 23
         2019,   # 24
         2019,   # 25
         2021,   # 26
         2022,   # 27
         2023,   # 28
         2023,   # 29
         2017,   # 30 (2017 à 2018)
         2018,   # 31 (2018 à 2019)
         2019,   # 32 (2019 à 2020)
         2020,   # 33 (2020 à 2021)
         2021,   # 34 (2021 à 2022)
         2022,   # 35 (2022 à 2023)
         2018,   # 36 (2018 à 2020)
         2023,   # 37
         2019,   # 38
         2023,   # 39
         2022,   # 40
         2018,   # 41 (2018 à 2020)
         2020,   # 42
         2020,   # 43
         2021,   # 44
         2021,   # 45
         2019,   # 46
         2020,   # 47
         2021,   # 48
         2021,   # 49
         2017,   # 50
         2019,   # 51
         2019,   # 52
         2020,   # 53
         2020,   # 54
         2023,   # 55
         2023,   # 56
         2023,   # 57
         2023,   # 58
         2023,   # 59
         2023,   # 60
         2020,   # 61
         2023,   # 62
         2023,   # 63
         2023,   # 64
         2023,   # 65
         2023,   # 66
         2023,   # 67
         2017,   # 68
         2018,   # 69
         2019,   # 70
         2020,   # 71
         2021,   # 72
         2022,   # 73
         2020,   # 74
         2020,   # 75
         2020,   # 76
         2021,   # 77
         2022,   # 78
         2021,   # 79
         2023,   # 80
         2023,   # 81
         2020,   # 82
         2020,   # 83
         2021,   # 84
         2021,   # 85
         2022,   # 86
         2022,   # 87
         2022,   # 88
         2022,   # 89
         2019,   # 90
         2020,   # 91
         2021,   # 92
         2021,   # 93
         2019,   # 94 (2019 à 2023)
         2022,   # 95
         2019,   # 96 (2019 à 2023)
         2020    # 97
    ],
    'Ano2': [
         0,    # 1
         0,    # 2
         0,    # 3
         2022, # 4 (2021-22)
         0,    # 5
         0,    # 6
         0,    # 7
         0,    # 8
         0,    # 9
         0,    # 10
         0,    # 11
         0,    # 12
         0,    # 13
         0,    # 14
         0,    # 15
         0,    # 16
         0,    # 17
         2021, # 18 (2017 à 2021)
         0,    # 19
         0,    # 20
         0,    # 21
         0,    # 22
         0,    # 23
         0,    # 24
         0,    # 25
         0,    # 26
         0,    # 27
         0,    # 28
         0,    # 29
         2018, # 30 (2017 à 2018)
         2019, # 31 (2018 à 2019)
         2020, # 32 (2019 à 2020)
         2021, # 33 (2020 à 2021)
         2022, # 34 (2021 à 2022)
         2023, # 35 (2022 à 2023)
         2020, # 36 (2018 à 2020)
         0,    # 37
         0,    # 38
         0,    # 39
         0,    # 40
         2020, # 41 (2018 à 2020)
         0,    # 42
         0,    # 43
         0,    # 44
         0,    # 45
         0,    # 46
         0,    # 47
         0,    # 48
         0,    # 49
         0,    # 50
         0,    # 51
         0,    # 52
         0,    # 53
         0,    # 54
         0,    # 55
         0,    # 56
         0,    # 57
         0,    # 58
         0,    # 59
         0,    # 60
         0,    # 61
         0,    # 62
         0,    # 63
         0,    # 64
         0,    # 65
         0,    # 66
         0,    # 67
         0,    # 68
         0,    # 69
         0,    # 70
         0,    # 71
         0,    # 72
         0,    # 73
         0,    # 74
         0,    # 75
         0,    # 76
         0,    # 77
         0,    # 78
         0,    # 79
         0,    # 80
         0,    # 81
         0,    # 82
         0,    # 83
         0,    # 84
         0,    # 85
         0,    # 86
         0,    # 87
         0,    # 88
         0,    # 89
         0,    # 90
         0,    # 91
         0,    # 92
         0,    # 93
         2023, # 94 (2019 à 2023)
         0,    # 95
         2023, # 96 (2019 à 2023)
         0     # 97
    ],
    'País_Moeda': [
         "US",  # 1 (USD)
         "GB",  # 2 (GBP)
         "GB",  # 3
         "GB",  # 4
         "GB",  # 5
         "GB",  # 6
         "GB",  # 7
         "GB",  # 8
         "GB",  # 9
         "GB",  # 10
         "GB",  # 11
         "GB",  # 12
         "GB",  # 13
         "GB",  # 14
         "GB",  # 15
         "GB",  # 16
         "GB",  # 17
         "GB",  # 18
         "GB",  # 19
         "GB",  # 20
         "GB",  # 21
         "GB",  # 22
         "US",  # 23 (USD)
         "GB",  # 24
         "GB",  # 25
         "GB",  # 26
         "GB",  # 27
         "IN",  # 28 (INR)
         "IN",  # 29
         "IN",  # 30
         "IN",  # 31
         "IN",  # 32
         "IN",  # 33
         "IN",  # 34
         "IN",  # 35
         "IN",  # 36
         "IN",  # 37
         "IN",  # 38
         "IN",  # 39
         "IN",  # 40
         "IN",  # 41
         "IN",  # 42
         "IN",  # 43
         "IN",  # 44
         "IN",  # 45
         "IN",  # 46
         "IN",  # 47
         "IN",  # 48
         "IN",  # 49
         "KR",  # 50 (KRW)
         "KR",  # 51
         "KR",  # 52
         "KR",  # 53
         "KR",  # 54
         "KR",  # 55
         "KR",  # 56
         "KR",  # 57
         "KR",  # 58
         "KR",  # 59
         "KR",  # 60
         "KR",  # 61
         "KR",  # 62
         "KR",  # 63
         "KR",  # 64
         "KR",  # 65
         "KR",  # 66
         "KR",  # 67
         "KR",  # 68
         "KR",  # 69
         "KR",  # 70
         "KR",  # 71
         "KR",  # 72
         "KR",  # 73
         "MX",  # 74 (MXN)
         "MX",  # 75
         "MX",  # 76
         "MX",  # 77
         "MX",  # 78
         "US",  # 79 (USD)
         "US",  # 80
         "US",  # 81
         "US",  # 82
         "US",  # 83
         "US",  # 84
         "US",  # 85
         "US",  # 86
         "US",  # 87
         "US",  # 88
         "US",  # 89
         "US",  # 90
         "US",  # 91
         "US",  # 92
         "US",  # 93
         "ZA",  # 94 (ZAR)
         "ZA",  # 95
         "ZA",  # 96
         "ZA"   # 97
    ],
    'País do Programa': [
         "CN",  # 1
         "GB",  # 2
         "GB",  # 3
         "GB",  # 4
         "GB",  # 5
         "GB",  # 6
         "GB",  # 7
         "GB",  # 8
         "GB",  # 9
         "GB",  # 10
         "GB",  # 11
         "GB",  # 12
         "GB",  # 13
         "GB",  # 14
         "GB",  # 15
         "GB",  # 16
         "GB",  # 17
         "GB",  # 18
         "GB",  # 19
         "GB",  # 20
         "GB",  # 21
         "GB",  # 22
         "GB",  # 23
         "GB",  # 24
         "GB",  # 25
         "GB",  # 26
         "GB",  # 27
         "IN",  # 28
         "IN",  # 29
         "IN",  # 30
         "IN",  # 31
         "IN",  # 32
         "IN",  # 33
         "IN",  # 34
         "IN",  # 35
         "IN",  # 36
         "IN",  # 37
         "IN",  # 38
         "IN",  # 39
         "IN",  # 40
         "IN",  # 41
         "IN",  # 42
         "IN",  # 43
         "IN",  # 44
         "IN",  # 45
         "IN",  # 46
         "IN",  # 47
         "IN",  # 48
         "IN",  # 49
         "KR",  # 50
         "KR",  # 51
         "KR",  # 52
         "KR",  # 53
         "KR",  # 54
         "KR",  # 55
         "KR",  # 56
         "KR",  # 57
         "KR",  # 58
         "KR",  # 59
         "KR",  # 60
         "KR",  # 61
         "KR",  # 62
         "KR",  # 63
         "KR",  # 64
         "KR",  # 65
         "KR",  # 66
         "KR",  # 67
         "KR",  # 68
         "KR",  # 69
         "KR",  # 70
         "KR",  # 71
         "KR",  # 72
         "KR",  # 73
         "MX",  # 74
         "MX",  # 75
         "MX",  # 76
         "MX",  # 77
         "MX",  # 78
         "US",  # 79
         "US",  # 80
         "US",  # 81
         "US",  # 82
         "US",  # 83
         "US",  # 84
         "US",  # 85
         "US",  # 86
         "US",  # 87
         "US",  # 88
         "US",  # 89
         "US",  # 90
         "US",  # 91
         "US",  # 92
         "US",  # 93
         "ZA",  # 94
         "ZA",  # 95
         "ZA",  # 96
         "ZA"   # 97
    ],
    'Fontes': [
        "Economist Intelligence Unit, 2020²",                           # 1
        "Reino Unido, 2023⁷",                                             # 2
        "Reino Unido, 2022⁸",                                             # 3
        "HM Treasury, 2021¹⁸",                                            # 4
        "HM Treasury, 2021¹⁸",                                            # 5
        "Reino Unido, 2018⁵",                                              # 6
        "Reino Unido, 2018⁵",                                              # 7
        "Reino Unido, 2018³",                                              # 8
        "Reino Unido, 2018³",                                              # 9
        "Reino Unido, 2018⁴",                                              # 10
        "UK Parliament, 2023¹⁶",                                           # 11
        "Kight Frank LLP, 2023¹⁷",                                         # 12
        "Reino Unido, 2023⁹",                                              # 13
        "HM Government, 20211⁶",                                           # 14
        "HM Government, 2021⁶",                                            # 15
        "HM Government, 2021⁶",                                            # 16
        "HM Government, 2021⁶",                                            # 17
        "Innovate UK, 2021⁵⁰",                                             # 18
        "Association of the British Pharmaceutical Industry, 2023¹⁰",       # 19
        "Reino Unido, 2018¹¹",                                             # 20
        "Reino Unido, 2018¹¹",                                             # 21
        "Office for Life Sciences, 2019¹²",                                # 22
        "Office for Life Sciences, 2019¹²",                                # 23
        "Office for Life Sciences, 2019¹³",                                # 24
        "Office for Life Sciences, 2019¹³",                                # 25
        "Departament for Bussines, Energy & Industrial Strategy, 2021¹⁴",   # 26
        "Innovative UK, 2022¹⁵",                                           # 27
        "Ministry of Micro,Small & Medium Enterprises, 2023 ⁵²",            # 28
        "Biotechnology Industry Research Assistance Council, 2025⁵⁵",       # 29
        "Biotechnology Industry Research Assistance Council, 2018⁵⁶",       # 30
        "Biotechnology Industry Research Assistance Council, 2020⁵⁷",       # 31
        "Biotechnology Industry Research Assistance Council, 2020⁵⁷",       # 32
        "Biotechnology Industry Research Assistance Council, 2022⁵⁸",       # 33
        "Biotechnology Industry Research Assistance Council, 2022⁵⁸",       # 34
        "Biotechnology Industry Research Assistance Council, 2024⁵⁹",       # 35
        "Índia, 2023²³",                                                  # 36
        "Índia, 2023²³",                                                  # 37
        "Ministry of Micro,Small & Medium Enterprises, 2019²²",             # 38
        "Ministry of Chemicals and Fertilizers, 2023 ⁵⁴",                   # 39
        "Ministry of Chemicals & Fertilizers\nDepartment of Pharmaceuticals, 2022⁵³",  # 40
        "Department of Pharmaceuticals, 2018²⁵",                           # 41
        "Departament of Pharmaceuticals, 2020¹⁹",                          # 42
        "Índia, 2020²³",                                                  # 43
        "Índia, 2021²³",                                                  # 44
        "Ministry of Commerce & Industry, 2021²¹",                          # 45
        "Ministry of Chemicals and Fertilizers, 2019⁵¹",                    # 46
        "Ministry of Chemicals and Fertilizers, 2022²⁰",                    # 47
        "Ministry of Chemicals and Fertilizers, 2022²⁴",                    # 48
        "Ministry of Chemicals and Fertilizers, 2023²⁶",                    # 49
        "Coréia do Sul, 2019²⁸",                                            # 50
        "Coréia do Sul, 2019²⁸",                                            # 51
        "Coréia do Sul, 2019²⁹",                                            # 52
        "Ministries of Health and Welfare, Science and ICT, and Trade, Industry and Energy, 2020³⁰",  # 53
        "Ministries of Health and Welfare, Science and ICT, and Trade, Industry and Energy, 2020³⁰",  # 54
        "Ministry of Health and Welfare, 2023³⁴",                           # 55
        "Ministry of Science and ICT, 2024³⁶",                              # 56
        "Emergency Ministerial Meeting on Economic Affairs, 2023³¹",         # 57
        "Ministry of Health and Welfare, 2023³²",                           # 58
        "Ministry of Health and Welfare, 2023³⁵",                           # 59
        "Ministry of Health and Welfare, 2023³⁵",                           # 60
        "International Bank for Reconstruction and Development / The World Bank, 2024²⁷",  # 61
        "Ministry of Health and Welfare, 2023³³",                           # 62
        "Ministry of Health and Welfare, 2023³³",                           # 63
        "Ministry of Health and Welfare, 2023³³",                           # 64
        "Ministry of Health and Welfare, 2023³³",                           # 65
        "Ministry of Health and Welfare, 2023³³",                           # 66
        "Ministry of Health and Welfare, 2023³³",                           # 67
        "International Bank for Reconstruction and Development / The World Bank, 202427",  # 68
        "International Bank for Reconstruction and Development / The World Bank, 202427",  # 69
        "International Bank for Reconstruction and Development / The World Bank, 202427",  # 70
        "International Bank for Reconstruction and Development / The World Bank, 202427",  # 71
        "International Bank for Reconstruction and Development / The World Bank, 202427",  # 72
        "International Bank for Reconstruction and Development / The World Bank, 202427",  # 73
        "Consejo Nacional de Ciencia y Tecnología, 2021³⁷",                # 74
        "Consejo Nacional de Ciencia y Tecnología, 2021³⁷",                # 75
        "Secretaría de Salud, 2021⁶¹",                                      # 76
        "Secretaría de Salud, 2022⁶²",                                      # 77
        "Secretaría de Salud, 2023⁶³",                                      # 78
        "The White House, 2021⁴⁵",                                          # 79
        "Congressional Research Service, 2023⁴⁶",                           # 80
        "US Department of Energy, 2023⁴⁹",                                  # 81
        "United States Government Accountability Office, 2020³⁸",           # 82
        "United States Government Accountability Office, 2020³⁸",           # 83
        "The White House, 2021⁴⁴",                                          # 84
        "The White House, 2021⁴⁴",                                          # 85
        "Medicines Manufacturing Industry Partnership, 2023⁴⁷",            # 86
        "Estados Unidos, 2022²³",                                           # 87
        "U.S. Department of Health and Human Services, 2022³⁹",             # 88
        "The White House, 2022⁴⁰",                                          # 89
        "Administration for Strategic Preparedness and Response, 2021⁴¹",     # 90
        "United States Government Accountability Office, 2020⁴²",           # 91
        "Congressional Research Service, 2021⁴³",                           # 92
        "Congressional Research Service, 2021⁴³",                           # 93
        "Department of Trade, Industry and Competition, 2024⁴⁸",            # 94
        "Department of Trade, Industry and Competition, 2024⁴⁸",            # 95
        "Department of Trade, Industry and Competition, 2024⁴⁸",            # 96
        "South African Medical Research Council (SAMRC), 2020⁶⁰"            # 97
    ],
}


#Check se todos dados estão com o mesmo número de entradas
print(len(dados['Programa']),
len(dados['Descrição']),
len(dados['País_Moeda']),
len(dados['País do Programa']),
len(dados['Ano']),
len(dados['Ano2']),
len(dados['Gasto_Local']))

df = pd.DataFrame(dados)

97 97 97 97 97 97 97


## Conversor

In [47]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar Função para juntar Ano 1 e 2
df = definir_periodo(df)

# Aplicar as novas funções para as colunas 'Gasto_Local' e 'Gasto_Convertido_Dolar_2024'
df['Gasto Local'] = df.apply(lambda row: formatar_valor_moeda_local(row['Gasto_Local'], row['País_Moeda']), axis=1)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Exibir o DataFrame com as colunas formatadas
df_grafico = df[['País do Programa', 'Programa', 'Período', 'Gasto_Convertido_Dolar_2024','Gasto Convertido US$ 2024']]
df[['País do Programa', 'Programa', 'Descrição', 'Período', 'Gasto Local', 'Gasto Convertido US$ 2024', 'Fontes']]

<ipython-input-34-fa0d8168377c>:62: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



,País do Programa,Programa,Descrição,Período,Gasto Local,Gasto Convertido US$ 2024,Fontes
0,CN,Healthy China 2030,Gasto governamental em P&D em biotecnologia*,2019,USD 291.00 bilhões,USD 357.07 bilhões,"Economist Intelligence Unit, 2020²"
1,GB,Advanced Manufacturing Plan,Montante total disponibilizado para financiame...,2023,GBP 4.50 bilhões,USD 6.94 bilhões,"Reino Unido, 2023⁷"
2,GB,Advanced Manufacturing Plan,Parte do plano destinada às indústrias das ciê...,2023,GBP 520.00 milhões,USD 802.12 milhões,"Reino Unido, 2022⁸"
3,GB,Build Back Better: Our Plan for Growth and Inn...,Não específico para indústria da saúde: invest...,2021 a 2022,GBP 14.60 bilhões,USD 25.08 bilhões,"HM Treasury, 2021¹⁸"
4,GB,Build Back Better: Our Plan for Growth and Inn...,Não específico para indústria da saúde: financ...,2021,GBP 800.00 milhões,USD 1.44 bilhões,"HM Treasury, 2021¹⁸"
...,...,...,...,...,...,...,...
92,US,OperationWarp Speed (OWS),Valor dos contratos governamentais federais pa...,2021,USD 951.80 milhões,USD 1.10 bilhões,"Congressional Research Service, 2021⁴³"
93,ZA,Reimagined Industrial Strategy,Financiamento industrial fornecido ao longo de...,2019 a 2023,ZAR 100.00 bilhões,USD 15.61 bilhões,"Department of Trade, Industry and Competition,..."
94,ZA,Reimagined Industrial Strategy,"Abertura da nova instalação Sappi-Saiccor, que...",2022,ZAR 7.70 bilhões,USD 1.14 bilhões,"Department of Trade, Industry and Competition,..."
95,ZA,Reimagined Industrial Strategy,A IDC facilitou atividades de localização e be...,2019 a 2023,ZAR 10.00 bilhões,USD 1.56 bilhões,"Department of Trade, Industry and Competition,..."


In [38]:
# Agrupa os gastos por Período + País do Programa
df_agrupado = agrupar_gastos(df)

# Converte a coluna de valores para numérico
df_agrupado['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_agrupado['Gasto_Convertido_Dolar_2024'], errors='coerce')

# Formata os valores como dólar
df_agrupado['Gasto_Convertido_Dolar_2024'] = df_agrupado['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Agrupa as fontes por Período + País do Programa
df_fontes_agrupadas = agrupar_fontes_periodo_pais(df)

# Junta as fontes no df_agrupado
df_agrupado = pd.merge(df_agrupado, df_fontes_agrupadas, on=['Período', 'País do Programa'], how='left')

# Visualiza o DataFrame final com fontes
df_agrupado

,Período,País do Programa,Gasto_Convertido_Dolar_2024,Fontes
0,2017,GB,USD 162.74 bilhões,"Office for Life Sciences, 2019¹²; Reino Unido,..."
1,2017,KR,USD 3.94 bilhões,"Coréia do Sul, 2019²⁸; International Bank for ..."
2,2017 a 2018,IN,USD 1.86 milhões,Biotechnology Industry Research Assistance Cou...
3,2017 a 2021,GB,USD 466.34 milhões,"Innovate UK, 2021⁵⁰"
4,2018,GB,USD 1.07 bilhões,"Reino Unido, 2018³; Reino Unido, 2018⁴; Reino ..."
5,2018,KR,USD 152.28 milhões,International Bank for Reconstruction and Deve...
6,2018 a 2019,IN,USD 3.43 milhões,Biotechnology Industry Research Assistance Cou...
7,2018 a 2020,IN,USD 328.13 milhões,"Department of Pharmaceuticals, 2018²⁵; Índia, ..."
8,2019,CN,USD 357.07 bilhões,"Economist Intelligence Unit, 2020²"
9,2019,GB,USD 9.07 bilhões,"Office for Life Sciences, 2019¹²; Office for L..."


In [39]:
# Gasto por país + fontes
df_pais = agrupar_gastos_pais(df)
df_pais['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_pais['Gasto_Convertido_Dolar_2024'], errors='coerce')
df_pais['Gasto_Convertido_Dolar_2024'] = df_pais['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

df_fontes_pais = agrupar_fontes_pais(df)
df_pais = pd.merge(df_pais, df_fontes_pais, on='País do Programa', how='left')

df_pais

,País do Programa,Gasto_Convertido_Dolar_2024,Fontes
0,CN,USD 357.07 bilhões,"Economist Intelligence Unit, 2020²"
1,GB,USD 213.86 bilhões,Association of the British Pharmaceutical Indu...
2,IN,USD 29.68 bilhões,Biotechnology Industry Research Assistance Cou...
3,KR,USD 497.15 bilhões,"Coréia do Sul, 2019²⁸; Coréia do Sul, 2019²⁹; ..."
4,MX,USD 39.05 bilhões,"Consejo Nacional de Ciencia y Tecnología, 2021..."
5,US,USD 1.99 trilhões,Administration for Strategic Preparedness and ...
6,ZA,USD 18.31 bilhões,"Department of Trade, Industry and Competition,..."


In [40]:
# Gasto por programa + país + fontes
df_prog = agrupar_gastos_programa(df)
df_prog['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_prog['Gasto_Convertido_Dolar_2024'], errors='coerce')
df_prog['Gasto_Convertido_Dolar_2024'] = df_prog['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)
df_prog = df_prog.sort_values(by='País do Programa')

df_fontes_prog = agrupar_fontes_programa(df)
df_prog = pd.merge(df_prog, df_fontes_prog, on='Programa', how='left')
df_prog = df_prog.sort_values(by=['País do Programa', 'Programa'], ascending=[True, True])

df_prog

,Programa,País do Programa,Gasto_Convertido_Dolar_2024,Fontes
0,Healthy China 2030,CN,USD 357.07 bilhões,"Economist Intelligence Unit, 2020²"
11,Advanced Manufacturing Plan,GB,USD 7.74 bilhões,"Reino Unido, 2022⁸; Reino Unido, 2023⁷"
10,Build Back Better: Our Plan for Growth and Inn...,GB,USD 26.51 bilhões,"HM Treasury, 2021¹⁸"
8,Industrial Strategy Challenge Fund: Healthy ag...,GB,USD 581.31 milhões,"Reino Unido, 2018⁵"
7,Industrial Strategy Challenge Fund: Leading-ed...,GB,USD 486.85 milhões,"Reino Unido, 2018³; Reino Unido, 2018⁴"
6,Life Sciences 2030 Skills Strategy,GB,USD 1.93 bilhões,"Kight Frank LLP, 2023¹⁷; UK Parliament, 2023¹⁶"
5,Life Sciences Innovative Manufacturing Fund,GB,USD 26.22 milhões,"Reino Unido, 2023⁹"
1,Life Sciences Vision,GB,USD 1.27 bilhões,"HM Government, 20211⁶; HM Government, 2021⁶"
3,Medicines Manufacturing Challenge,GB,USD 466.34 milhões,"Innovate UK, 2021⁵⁰"
2,Medicines Manufacturing Industry Partnership (...,GB,USD 1.70 bilhões,Association of the British Pharmaceutical Indu...


In [44]:
df_grafico

,País do Programa,Programa,Período,Gasto_Convertido_Dolar_2024
0,CN,Healthy China 2030,2019,3.570708e+11
1,GB,Advanced Manufacturing Plan,2023,6.941410e+09
2,GB,Advanced Manufacturing Plan,2023,8.021185e+08
3,GB,Build Back Better: Our Plan for Growth and Inn...,2021 a 2022,2.507856e+10
4,GB,Build Back Better: Our Plan for Growth and Inn...,2021,1.436131e+09
...,...,...,...,...
92,US,OperationWarp Speed (OWS),2021,1.101893e+09
93,ZA,Reimagined Industrial Strategy,2019 a 2023,1.560967e+10
94,ZA,Reimagined Industrial Strategy,2022,1.143452e+09
95,ZA,Reimagined Industrial Strategy,2019 a 2023,1.560967e+09


In [52]:
# prompt: gráfico plotly de Gasto_Convertido_Dolar_2024 num groupby por País do Programa

import plotly.express as px

# Agrupa por 'País do Programa' e soma 'Gasto_Convertido_Dolar_2024'
df_grafico_agrupado = df_grafico.groupby('País do Programa')['Gasto_Convertido_Dolar_2024'].sum().reset_index()

# Cria o gráfico de barras com o DataFrame agrupado
fig = px.bar(df_grafico_agrupado,
             x='País do Programa',
             y='Gasto_Convertido_Dolar_2024',
             color='País do Programa',
             title='Gasto Convertido por País do Programa (2024)',
             labels={'Gasto_Convertido_Dolar_2024': 'Gasto Convertido (US$)'},
             template='plotly_white',
             text='Gasto_Convertido_Dolar_2024')

fig.update_traces(texttemplate='%{y:.3s}', textposition='outside') # Modifica o texttemplate
fig.show()
